# 2024-1 SCSC SIG: Fluent Python
## Week 1 Discussion: Part 1 - Data Structures
### Chapter 2 - An Array of Sequences
Date: 2024-04-05

Speaker: Samuel Seungsup Lee, Mechanical and Aerospace Engineering, Seoul National University

### 0. Before Reading: Background Knowledge

**Container sequence**
: 모든 객체를 담을 수 있는 시퀀스. 예로 list, tuple, collections.deque 등이 있다. 강력하지만 크고 둔하다.

**Flat sequence**
: 단순자료형만 담을 수 있는 시퀀스. 예로 str, bytes, array.array 등이 있다. 단순하지만 작고 빠르다.

**Mutable sequence**
: (아마?) 안에 담은 데이터를 직접적으로 수정할 수 있는 시퀀스. 예로 list, bytearray, array.array, collections.deque 등이 있다.

**Immutable sequence**
: 안에 담은 데이터를 직접적으로 수정할 수 없는 시퀀스. 예로 str, tuple, bytes 등이 있다. (str을 immutable하게 설정해서 효율적인 메모리 할당, dict의 key로 활용 등이 가능해졌다.)

**List comprehension**
: `[foo(dat) for dat in dats]`

**Walrus operator**
: Comprehension이나 expression 안에 `:=`식으로 정의된 변수는 연산 종료 이후에도 호출이 가능하다. 

**Syntax Tips**
: [], {},, () 안에 줄바꿈은 무시되고, 마지막 항 이후 쉼표 `,`도 무시된다. 일반적으로 마지막 항 이후에도 쉼표를 넣어서 타 개발자가 항목을 추가했을 때 변경항목의 개수를 줄이는 것이 예의이다. Tuple에 원소가 하나만 있으면 (dat,)식으로 쉼표 `,`을 반드시 넣어야 한다.

**Tuples**
: Tuple의 장점으로는 길이가 절대로 변하지 않는다는 것과 list보다 메모리 사용이 적은 것이 있다. Tuple을 생성할 때는 하나의 작업으로 byte가 생성되지만, list을 생성할 때는 stack으로 각 원소의 byte를 넣은 후에 list을 생성한다. t가 tuple이라면 tuple(t)는 해당 t를 바로 reference하지만, l이 list더라도 list(l)은 l의 copy를 생성한다. list는 mutable하므로 여유메모리를 필요로 하지만, tuple은 여유메모리가 필요 없다. Tuple 내 원소의 reference는 tuple struct안에 바로 저장이 되지만, list는 다른 주소에서 저장된 reference array를 가리키는 pointer가 저장되어서 상대적으로 비효율적이다. 

**Slice**
: `[a:b:c]`에서 `c`는 몇 개씩 자를 건지를 나타내는 stride이다. 음수로 설정하면 reverse해서 slice할 수 있다. 

**Deque**
`collections.deque`는 양단에서 원소 삽입/삭제가 가능한 빠른 `queue` 객체이다. `maxlen=` parameter로 길이를 지정할 수 있어서 새 원소를 추가할 때 자동으로 반대단 원소가 삭제되게 할 수 있다. 

### 1. Topics Covered

1. List comprehensions and the basics of generator expressions.

2. Using tuples as records versus using tuples as immutable lists.

3. Sequence unpacking and sequence patterns.

4. Reading from slices and writing to slices.

5. Specialized sequence types, like arrays and queues.

#### 1.1. List Comprehensions and Generator Expressions

#### Insights
1. List comprehension은 list을 만드는 강력한 도구이다. List 생성이 아닌 다른 부가효과를 얻고 싶을 경우에는 readability를 위해 for loop을 쓰자.
2. List comprehension은 filter, map보다 빠를 수도 있다.

In [4]:
# Fluent Python code repository: 02-array-seq/listcomp_speed.py

import timeit

TIMES = 10000

SETUP = """
symbols = '$¢£¥€¤'
def non_ascii(c):
    return c > 127
"""

def clock(label, cmd):

    res = timeit.repeat(cmd, setup=SETUP, number=TIMES)

    print(label, *(f'{x:.3f}' for x in res))

clock('listcomp        :', '[ord(s) for s in symbols if ord(s) > 127]')

clock('listcomp + func :',
      '[ord(s) for s in symbols if non_ascii(ord(s))]')

clock('filter + lambda :', 'list(filter(lambda c: c > 127, map(ord, symbols)))')

clock('filter + func   :', 'list(filter(non_ascii, map(ord, symbols)))')

listcomp        : 0.018 0.012 0.020 0.019 0.020
listcomp + func : 0.030 0.034 0.019 0.026 0.019
filter + lambda : 0.017 0.016 0.018 0.018 0.014
filter + func   : 0.012 0.016 0.014 0.020 0.024


3. List가 아니라 단순히 iterate이 가능한 원소들이 필요할 경우 *genexp* (generator expression, 생성형 표현?)을 사용하면 원소 하나를 만들고 지우기를 반복해 list을 생성하는데 사용될 메모리를 아낀다. `(foo(dat) for dat in dats)`처럼 표현식을 괄호로 두르면 된다. 함수가 genexp을 유일한 변수로 받을 경우 굳이 괄호를 칠 필요는 없다.

### 1.2 Using tuples as records versus using tuples as immutable lists.

4. Tuple을 단순히 immutable list가 아니라, field name이 없는 기록자료로도 쓸 수 있다. 예로 여행한 국가들을 순서대로 tuple에 저장한다면, tuple의 순서와 tuple 안 원소의 개수가 매우 중요해진다.

5. Tuple의 원소에 mutable object을 저장한다면 그 object가 변경될 경우 tuple도 변경된다. 일반적으로 tuple 안에 mutable object을 원소로 저장하지 않는다.


### 1.3. Sequence unpacking and sequence patterns.

6. Tuple을 unpack할 때 (요즘에는 iterable unpacking이라고 부르는 듯) a, b, c = (a_val, b_val, c_val) 식으로 바로 할 수 있다. 또한 print문에서도 %s%s%s 식으로 바로 unpacking이 가능하다. Unpacking을 이용해서 `a,b=b,a`식으로 빠른 순서 바꾸기도 가능하다. Nested expression도 형식만 일관적이면 unpacking이 가능하다.

In [ ]:
# Example 2-10. Destructuring nested tuples.

metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]


def main():
    print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
    for record in metro_areas:
        match record:
            case [str(name), _, _, (float(lat), float(lon))] if lon <= 0:
                # str(name), *_, (float(lat), float(lon)) 으로도 작성 가능
                print(f'{name:15} | {lat:9.4f} | {lon:9.4f}')

7. Match/case을 이용해 웬만한 시퀀스를 빠르게 분류하고 명령을 실행할 수 있다. Nested 시퀀스를 match할 경우 []와 ()는 차이가 없다. 단 match에서 str, byte, bytearray는 시퀀스로 인식되지 않는다. 인식하고자 한다면 tuple(dat)으로 바꿔야 한다.

In [ ]:
# Example 2-11. Matching patterns without `match/case`
def evaluate(exp: Expression, env: Environment) -> Any:
    "Evaluate an expression in an environment."
    if isinstance(exp, Symbol): # variable reference
        return env[exp]
# ... lines omitted
    elif exp[0] == 'quote': # (quote exp)
        (_, x) = exp
        return x
    elif exp[0] == 'if': # (if test conseq alt)
        (_, test, consequence, alternative) = exp
        if evaluate(test, env):
            return evaluate(consequence, env)
        else:
            return evaluate(alternative, env)
    elif exp[0] == 'lambda': # (lambda (parm…) body…)
        (_, parms, *body) = exp
        return Procedure(parms, body, env)
    elif exp[0] == 'define':
        (_, name, value_exp) = exp
        env[name] = evaluate(value_exp, env)
# ... more lines omitted

In [ ]:

# Example 2-12. Pattern matching with `match/case` (Python>=3.10)

def evaluate(exp: Expression, env: Environment) -> Any:
    "Evaluate an expression in an environment."
    match exp:
    # ... lines omitted
    case ['quote', x]:
        return x
    case ['if', test, consequence, alternative]:
        if evaluate(test, env):
            return evaluate(consequence, env)
        else:
            return evaluate(alternative, env)
    case ['lambda', [*parms], *body] if body:
        return Procedure(parms, body, env)
    case ['define', Symbol() as name, value_exp]:
        env[name] = evaluate(value_exp, env)
    # ... more lines omitted
    case _:
        raise SyntaxError(lispstr(exp))

### 1.4. Reading from slices and writing to slices.

8. Slicing할 때 생성되는 slice(start,stop,step) 객체를 명명하고 반복해서 사용할 수 있다. 예로 `THREES=slice(1,100,3)`를 정의하면 `l[THREE]`는 `l[1:100:3]`과 동일하다.

9. Slice에서 `x[i,...]`은 `x[i,:,:,:,]`와 동일하다. 이외에도 여러 가지 재밌는 작업이 가능하다.

In [ ]:
# Assigning to Slices

l=list(range(10))
print(l)
l[2:5]=[20,30]
print(l)
del l[5:7]
print(l)
l[3::2]=[11,22]
print(l)
l[2:5]=[100]     #l[2:5]=100 은 TypeError
print(l)

10. 시퀀스에서 +, *는 새로운 객체를 생성한다. 이 때 nested list을 생성할 경우 `nl=[['_']*3 for i in range(3)]`은 정상적으로 nl을 생성하지만, `nl=[['_']*3]*3`은 한 row를 바꾸면 모든 row가 바뀌는 결과가 나온다.

11. `+=`, `*=`를 사용하면 list는 같은 list가 변경되지만 tuple은 새로운 tuple이 copy+cat되어 생성된다. Immutable 시퀀스에 `+=`, `*=` 등을 여러 번 시행할 경우 메모리 낭비가 심하므로 자제하는 것이 좋다.

12. Method가 해당 객체를 직접 변경할 경우 항상 `None`을 return한다. 이를 통해 copy를 생성하는지, 직접 변경하는지 파악할 수 있다. 

### 1.5. Specialized sequence types, like arrays and queues.


13. 백만 단위의 float을 사용할 경우 `array.array`를, 처음과 마지막에 새로운 원소를 지속적으로 추가/변경/제거할 경우 `collections.deque`를 쓰는 것이 좋다. 

14. 특히 `array.array`는 C만큼 효율적이므로 `.frombytes`, `.tofile` 등 효율적인 저장/불러오기도 가능하다. 대신 `.sort()`는 없으므로 `sorted(arr)`을 사용하고, sorted 상태를 유지하면서 새 원소를 추가해야 한다면 `bisect.insort(arr, val)`을 사용하는 것이 좋다.

15. Array를 사용할 때 memoryview를 이용하면 NumPy와 비슷하게 array 수정이 가능하다. bit 이동이 없으므로 같은 객체가 계속 수정이 된다.

In [ ]:
# Example 2-20. Handling 6 bytes of memory such as 1x6, 2x3, and 3x2 views.

from array import array
octets = array('B', range(6))
m1=memoryview(octets)
print(f"m1: {m1.tolist()}")
m2=m1.cast('B', [2,3])
print(f"m2: {m2.tolist()}")
m3=m1.cast('B', [3,2])
print(f"m3: {m3.tolist()}")
m2[1,1]=22
m3[1,1]=33
print(f"octets: {octets}")

16. Deque 외에 queue (SimpleQueue, Queue, LifoQueue, PriorityQueue), multiprocessing (unbounded SimpleQueue, bounded Queue), asyncio (Queue, LifoQueue, PriorityQueue, JoinableQueue), heapq (heappush, heappop).

### Any Questions?